# 📘 Sesión 7: Comparación de Modelos e Introducción a Redes Neuronales


## 🎯 Objetivo
Comparar modelos supervisados e introducir redes neuronales simples con `MLPClassifier`.

## 🔍 Carga de datos
Usaremos el dataset `breast_cancer` de sklearn.

In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## ⚙️ Preparación de datos

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 📊 Comparación de modelos

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

models = {
    'KNN': KNeighborsClassifier(),
    'Regresión Logística': LogisticRegression(max_iter=300),
    'Árbol de Decisión': DecisionTreeClassifier()
}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{name}: Exactitud promedio (cross-val): {scores.mean():.3f} ± {scores.std():.3f}")


KNN: Exactitud promedio (cross-val): 0.955 ± 0.017
Regresión Logística: Exactitud promedio (cross-val): 0.957 ± 0.006
Árbol de Decisión: Exactitud promedio (cross-val): 0.910 ± 0.029


### 📘 Explicación del código

- `from sklearn.neighbors import KNeighborsClassifier`, `LogisticRegression`, `DecisionTreeClassifier`:  
  Se importan tres modelos supervisados clásicos desde `scikit-learn` para comparación.

- `from sklearn.model_selection import cross_val_score`:  
  Importa la función que permite aplicar validación cruzada a un modelo. Evalúa el desempeño promedio de un modelo entrenado y validado en múltiples particiones del conjunto de datos.

- `models = { ... }`:  
  Se define un diccionario con los tres modelos: KNN, regresión logística y árbol de decisión, cada uno instanciado con sus parámetros por defecto (excepto `max_iter=300` en regresión logística para asegurar convergencia).

- `for name, model in models.items()`:  
  Itera sobre cada modelo definido para evaluarlo con la misma estrategia.

- `cross_val_score(...)`:  
  Ejecuta validación cruzada con 5 particiones (`cv=5`) sobre los datos de entrenamiento. Se mide la `accuracy` en cada iteración.

- `scores.mean()`, `scores.std()`:  
  Calcula el promedio y la desviación estándar de las 5 mediciones de exactitud obtenidas por la validación cruzada.

Este enfoque permite evaluar los modelos de forma justa, reduciendo el riesgo de sobreajuste a una sola división de los datos.


### 📊 Análisis de resultados de validación cruzada

| Modelo               | Exactitud promedio | Desviación estándar |
|----------------------|--------------------|----------------------|
| KNN                  | 0.955              | ± 0.017              |
| Regresión Logística  | 0.957              | ± 0.006              |
| Árbol de Decisión    | 0.909              | ± 0.013              |

- **KNN y Regresión Logística** muestran un rendimiento muy alto y similar en precisión promedio (~95.5%), con la regresión ligeramente mejor.
- La **regresión logística** además tiene una **desviación estándar más baja**, lo que indica que su desempeño es más estable entre particiones del conjunto de datos.
- El **árbol de decisión** tiene una exactitud promedio menor (90.9%) y una desviación más alta que la regresión, lo que sugiere mayor variabilidad y posiblemente más sensibilidad a los datos de entrenamiento.

**Conclusión:** En este conjunto de datos, tanto KNN como regresión logística generalizan bien, pero la regresión muestra más consistencia. El árbol podría estar sobreajustando o no capturando adecuadamente la estructura de los datos.


## 🤖 Introducción a redes neuronales (MLPClassifier)

In [4]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,), activation='relu', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

mlp_score = mlp.score(X_test, y_test)
print(f"Exactitud del MLPClassifier: {mlp_score:.3f}")

Exactitud del MLPClassifier: 0.977


### 📘 Explicación del código

- `from sklearn.neural_network import MLPClassifier`:  
  Importa la clase `MLPClassifier`, que implementa un perceptrón multicapa (red neuronal artificial) para tareas de clasificación supervisada.

- `mlp = MLPClassifier(...)`:  
  Se crea un modelo de red neuronal con los siguientes parámetros:
  - `hidden_layer_sizes=(20,)`: se define una sola capa oculta con 20 neuronas.
  - `activation='relu'`: se utiliza la función de activación ReLU, común en redes profundas por su eficiencia.
  - `max_iter=1000`: se permite hasta 1000 iteraciones para que el modelo converja.
  - `random_state=42`: se fija una semilla para reproducibilidad.

- `mlp.fit(X_train, y_train)`:  
  Entrena la red neuronal con los datos normalizados de entrenamiento. Durante el entrenamiento, la red ajusta los pesos y sesgos internos para minimizar el error de clasificación.

- `mlp.score(X_test, y_test)`:  
  Calcula la exactitud del modelo sobre el conjunto de prueba, es decir, la proporción de predicciones correctas.

- `print(...)`:  
  Muestra la exactitud alcanzada por el modelo entrenado sobre datos no vistos.


### 📊 Análisis ampliado del resultado

El modelo `MLPClassifier` alcanzó una exactitud del **97.7%** sobre el conjunto de prueba. Este desempeño es superior al reportado por los modelos tradicionales comparados mediante validación cruzada:

- **KNN**: 95.5%
- **Regresión logística**: 95.7%
- **Árbol de decisión**: 90.9%

Aunque esta comparación debe tomarse con precaución —ya que el MLP fue evaluado sobre una única partición de prueba (`test set`) mientras los otros modelos fueron evaluados con validación cruzada (`cross_val`)—, el resultado sugiere que la red neuronal fue capaz de **capturar relaciones más complejas** en los datos que los otros algoritmos no lograron modelar completamente.

Este buen desempeño puede atribuirse a varias características de las redes neuronales:

- Son capaces de **modelar no linealidades** de forma natural gracias a sus funciones de activación no lineales.
- Pueden **componer transformaciones internas** a través de múltiples capas, lo que les permite aprender representaciones jerárquicas de los datos.
- El entrenamiento se realiza optimizando una función de pérdida global, lo que favorece una solución más ajustada a la estructura de los datos.

No obstante, las redes neuronales también presentan **ciertas limitaciones importantes**:

- Son modelos que requieren **más tiempo de entrenamiento**, especialmente si el número de capas o de neuronas es elevado.
- Son **menos interpretables**, ya que no generan reglas claras ni coeficientes con sentido directo, como en la regresión logística o los árboles de decisión.
- Son **sensibles a los hiperparámetros** y a la inicialización, lo que hace necesario ajustar cuidadosamente valores como el número de capas, el tamaño de cada capa, el tipo de activación o el algoritmo de optimización.

Por estas razones, aunque el resultado obtenido es muy positivo, se recomienda complementar este análisis con otras métricas como la **precisión, recall, F1-score** y especialmente con la **matriz de confusión**, para verificar que el buen rendimiento global no oculte deficiencias graves en alguna de las clases (por ejemplo, un mal desempeño al identificar la clase minoritaria).

Además, puede ser útil aplicar **validación cruzada también al MLPClassifier**


### ✅ Conclusión

El modelo `MLPClassifier` logró un desempeño superior en términos de exactitud (97.7%) frente a los modelos comparados previamente. Aunque esta cifra proviene de una evaluación directa sobre el conjunto de prueba (y no de validación cruzada), sugiere que la red neuronal fue capaz de modelar relaciones más complejas en los datos.

Esto resalta el potencial de las redes neuronales cuando existen patrones no lineales difíciles de capturar con modelos más simples. Sin embargo, su implementación también implica desafíos: mayor tiempo de entrenamiento, menor interpretabilidad y sensibilidad a la configuración de hiperparámetros.

Por tanto, si bien su rendimiento es prometedor, se recomienda validar su comportamiento con métricas adicionales como F1-score, AUC y matriz de confusión. También sería ideal aplicar validación cruzada para obtener una comparación equitativa con los modelos previos y garantizar su estabilidad frente a nuevas particiones del conjunto de datos.
